In [1]:
using Random
using NNlib: conv, maxpool
using Flux
Random.seed!(1234);

function conv_nowe(I, K)
    H, W, C, N = size(I) 
    HH, WW, C, F = size(K)
    H_R = 1 + H - HH
    W_R = 1 + W - WW
    out = zeros(H_R, W_R, F, N)
    for n=1:N
        for depth=1:F
            @views for r=1:H_R
                for c=1:W_R
                    out[r, c, depth, n] = sum(I[r:r+HH-1, c:c+WW-1, :, n] .* K[:, :, :, depth]) 
                end
            end
        end
    end
    return out
end


function conv_noweP(I, K)
    #I = reshape(I, 28 , 28, 1, 1)
    H, W, C, N = size(I) 
    HH, WW, C, F = size(K)
    H_R = 1 + H - HH
    W_R = 1 + W - WW
    out = zeros(H_R, W_R, F, N)
    for r=1:H_R
        @views for c=1:W_R
            r_field = I[r:r+HH-1, c:c+WW-1, :, :]
            r_field_flat = reshape(r_field, HH*WW*C, N)
            K_flat = reshape(K, HH*WW*C, F)
            out[r, c, :, :] = sum(K_flat .* r_field_flat, dims = 1)
        end
    end
    return out
end

function create_kernels_nowe(kernel_height, kernel_width, n_input, n_output)
    # Inicjalizacja Xaviera
    squid = sqrt(6 / (n_input + n_output * (kernel_width * kernel_height)))
    random_vals = randn(kernel_height, kernel_width, n_input, n_output) * squid
    return random_vals
end

function create_kernels(n_input::Int64, n_output::Int64, kernel_width::Int64, kernel_height::Int64)
    # Inicjalizacja Xaviera
    squid::Float32 = sqrt(6 / (n_input + n_output * (kernel_width * kernel_height)))
    random_vals = randn(Float32, kernel_height, kernel_width, n_input, n_output) * squid
    return random_vals
end

create_kernels (generic function with 1 method)

In [2]:
test_n = Float32.([0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.21568628 0.53333336 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.6745098 0.99215686 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.07058824 0.8862745 0.99215686 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.19215687 0.07058824 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.67058825 0.99215686 0.99215686 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.11764706 0.93333334 0.85882354 0.3137255 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.09019608 0.85882354 0.99215686 0.83137256 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.14117648 0.99215686 0.99215686 0.6117647 0.05490196 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.25882354 0.99215686 0.99215686 0.5294118 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.36862746 0.99215686 0.99215686 0.41960785 0.003921569 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.09411765 0.8352941 0.99215686 0.99215686 0.5176471 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.6039216 0.99215686 0.99215686 0.99215686 0.6039216 0.54509807 0.043137256 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.44705883 0.99215686 0.99215686 0.95686275 0.0627451 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.011764706 0.6666667 0.99215686 0.99215686 0.99215686 0.99215686 0.99215686 0.74509805 0.13725491 0.0 0.0 0.0 0.0 0.0 0.15294118 0.8666667 0.99215686 0.99215686 0.52156866 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07058824 0.99215686 0.99215686 0.99215686 0.8039216 0.3529412 0.74509805 0.99215686 0.94509804 0.31764707 0.0 0.0 0.0 0.0 0.5803922 0.99215686 0.99215686 0.7647059 0.043137256 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07058824 0.99215686 0.99215686 0.7764706 0.043137256 0.0 0.007843138 0.27450982 0.88235295 0.9411765 0.1764706 0.0 0.0 0.18039216 0.8980392 0.99215686 0.99215686 0.3137255 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07058824 0.99215686 0.99215686 0.7137255 0.0 0.0 0.0 0.0 0.627451 0.99215686 0.7294118 0.0627451 0.0 0.50980395 0.99215686 0.99215686 0.7764706 0.03529412 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.49411765 0.99215686 0.99215686 0.96862745 0.16862746 0.0 0.0 0.0 0.42352942 0.99215686 0.99215686 0.3647059 0.0 0.7176471 0.99215686 0.99215686 0.31764707 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.53333336 0.99215686 0.9843137 0.94509804 0.6039216 0.0 0.0 0.0 0.003921569 0.46666667 0.99215686 0.9882353 0.9764706 0.99215686 0.99215686 0.7882353 0.007843138 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.6862745 0.88235295 0.3647059 0.0 0.0 0.0 0.0 0.0 0.0 0.09803922 0.5882353 0.99215686 0.99215686 0.99215686 0.98039216 0.30588236 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.101960786 0.6745098 0.32156864 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.105882354 0.73333335 0.9764706 0.8117647 0.7137255 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.6509804 0.99215686 0.32156864 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2509804 0.007843138 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.9490196 0.21960784 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.96862745 0.7647059 0.15294118 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.49803922 0.2509804 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;;;]);
jj = create_kernels(1, 6, 3, 3);

In [3]:
using LinearAlgebra
function im2col(I::Array{Float32, 4}, KH::Integer, KW::Integer, OH::Integer, OW::Integer)
    IH, IW, IC, IN = size(I)
    col = Array{Float32}(undef, KH * KW * IC, OH * OW * IN)
    idx = 1
    for n in 1:IN
        for j in 1:OH
            for i in 1:OW
                @inbounds @views patch =  I[j:j+KH-1, i:i+KW-1, :, n]
                col[:, idx] = reshape(patch, KH * KW * IC)
                idx += 1
            end
        end
    end
    return col
end

function conv_fast2(I::Array{Float32, 4}, K::Array{Float32, 4})
    KH, KW, KIC, KOC = size(K)
    IH, IW, IC, IN = size(I)
    H_O = 1 + IH - KH
    W_O = 1 + IW - KW
    col_I = im2col(I, KH, KW, H_O, W_O)
    col_K = transpose(reshape(K, :, KOC))
    conv = zeros(Float32, KOC, H_O * W_O * IN)
    mul!(conv, col_K, col_I)
    output = reshape(conv, H_O, H_O, KOC, IN)
    #output = permutedims(output, (2, 3, 1, 4))
    return output
end


conv_fast2 (generic function with 1 method)

In [4]:
@benchmark conv_fast2(test_n, jj)

LoadError: LoadError: UndefVarError: `@benchmark` not defined
in expression starting at d:\Studia\Chabrystyka\konwolucje.ipynb:1

In [5]:
using BenchmarkTools
@benchmark conv_fast2(test_n, jj)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  44.300 μs …   5.111 ms  ┊ GC (min … max): 0.00% … 98.47%
 Time  (median):     56.600 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   59.784 μs ± 134.131 μs  ┊ GC (mean ± σ):  6.63% ±  2.94%

   █▇▂             ▃▃▃                                          
  ▂████▅▅▃▃▂▂▂▂▃▅▅█████▇▆▅▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  44.3 μs         Histogram: frequency by time         89.1 μs <

 Memory estimate: 40.08 KiB, allocs estimate: 7.

In [6]:
using BenchmarkTools
@benchmark conv(test_n, jj)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   7.000 μs …  1.853 ms  ┊ GC (min … max):  0.00% … 98.64%
 Time  (median):     18.350 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   20.649 μs ± 76.680 μs  ┊ GC (mean ± σ):  22.33% ±  6.01%

  ▂█                    ▅▅▃                                    
  ██▅▄▃▂▂▂▂▁▂▁▁▁▁▁▁▂▃▂▂▄███▇▄▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  7 μs            Histogram: frequency by time        36.7 μs <

 Memory estimate: 43.22 KiB, allocs estimate: 39.

In [9]:
g = Float32[-0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.049749225 0.08635205 0.0 -0.23352845 0.0 0.2297098 0.0 0.0; 0.18651241 0.0 0.0 0.10113871 0.0 0.0 0.0 -0.016092606 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.07598951 0.0 0.0 0.28360012 0.0 0.0; 0.0 0.0 0.0 0.33505672 0.72186625 0.0 0.0 0.24543135 0.0 0.0 0.0; 0.046354827 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.32792395 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.32778266 0.0 0.1730154 0.0 0.0 -0.005417116 0.0 -0.12249918 -0.50731266 0.0 0.0; -0.049786806 0.0 0.0 0.0 0.0 0.0 0.10101655 0.0 0.06997129 0.0 0.0; 0.0 0.0 0.016544707 0.0 0.0 -0.076849304 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 -0.76895493 -0.008114379 0.0 0.08621851 0.0 0.0; 0.0 0.0 0.0 0.029211536 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.07013579 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; -0.023687199 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.22321387 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.15890315 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.21567445 0.25926065 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.50576204 0.0 0.55246985 0.0 0.0 0.0 0.0; -0.15415977 0.0 0.0 -0.2089773 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.044110104 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 -0.24008916 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.25650623 0.13374916 0.0 0.0 -0.036169995 -0.23757733 0.0 0.0; -0.0 0.0 0.0 -0.023198128 0.16416907 0.0 0.47243533 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.17592815 -0.11703283 0.0 0.0 -0.050258867 0.0 0.281806 -0.21538556 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.023846786 0.08183714 0.0 0.0 -0.37145853 0.24890672 0.0 0.0; 0.0 0.0 0.0 0.08574329 -0.040734395 0.0 -0.0 0.0 -0.19770399 0.0 0.0; -0.03002809 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.2872743 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.038403258 -0.18088162 0.0 -0.20643881 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.16196246 0.0 -0.0 0.0 0.0; 0.0 0.1677809 0.0 -0.26607448 -0.20912428 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.2186228 -0.3138034 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.044034965 -0.0136195645 0.0 0.0 0.07122475 0.23994167 0.0 0.0; 0.0 0.0 0.0 0.4592989 0.0 0.045803517 -0.14750506 0.0 0.0 0.0 0.0; 0.36943108 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.028137013 0.0 0.0; 0.0 0.0 0.0 0.0 0.03631328 0.0 0.0 0.0 -0.27467763 0.0 0.0; -0.36708343 0.0 -0.16603279 0.0 0.0 0.0 0.0 0.3780577 0.0 0.0 0.0; 0.16428426 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.076784365 0.0 0.0; 0.0 0.0 0.013486713 0.0 0.0 0.36636782 0.0 0.27550012 0.0 0.0 0.0; 0.0 0.040085927 0.0 0.0 0.0 0.15056658 0.3983512 0.0 0.20393862 0.0 0.0; 0.0 0.0 -0.05470155 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.43343097 -0.3450163 0.0 -0.21355045 0.0 -0.14461808 0.0 0.0; 0.0 0.0 0.0 -0.75233585 0.0 0.11660064 0.0 0.18088934 0.26855022 0.0 0.0; 0.0 -0.020664334 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.48381552 0.0 0.0 0.0 0.0 -0.051790863 0.0; -0.23524842 0.0 0.24303122 0.0 0.0 0.0 0.0 -0.094092205 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.08472664 0.0 0.0; 0.0 -0.045888335 0.19836825 0.0 0.0 -0.26804534 0.0 -0.08128792 0.0 0.0 0.0; 0.0 0.48730505 -0.08708805 0.0 0.0 -0.096306354 0.0 0.16878279 -0.5288198 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.07377343 -0.1556396 0.0 0.21488506 0.0 0.3633014 0.0 0.0; 0.0 0.0 0.0 0.05210159 -0.100083925 0.0 -0.3522511 0.0 0.19234417 0.0 0.0; 0.0 0.28329307 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.10968843 0.0 0.0 -0.07269997 0.0 0.0; -0.3280528 0.0 0.33488086 0.0 0.0 0.0 0.0 -0.007189691 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.025387853 0.0 0.0; 0.3619783 0.0 0.24938218 0.0 0.04257442 0.0 0.0 0.36859402 0.0 0.0 0.0; -0.10996839 0.0 0.26533186 0.0 0.0 0.0 -0.58297986 0.0 0.37198728 0.0 0.0; 0.0 0.0 0.0 0.0 -0.13055512 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.057250198 0.0 0.0 -0.14909537 -0.19915414 0.0 0.0 0.0 0.0; 0.05629429 0.0 0.0 0.0 0.0 0.45008457 0.0 0.42223644 0.0 0.0 0.0; 0.0 0.0 0.0 0.23686668 0.0 0.0 0.0 0.0 0.065072045 0.0 0.0; 0.0 0.0 0.0 0.2822465 -0.23639835 0.0 0.0 -0.20673093 0.0 0.0 0.0; -0.6275462 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.04498879 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.2718905 0.0 0.0; 0.7024522 0.0 0.0 0.0 0.0 0.0 0.0 0.063813284 0.0 0.0 0.0; 0.0 -0.17351499 0.0 0.0 0.0 0.0 0.05809015 0.0 -0.073942766 0.0 0.0; 0.0 0.0 0.0 -0.23382217 -0.2506652 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.30877352 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 -0.08134182 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2.30968f-5 -0.2452333 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.23581699 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.2868507 0.2117471 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.25961164 0.30143496 0.0 -0.0 0.0 0.0 0.0 0.0; 0.36906564 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1472209 0.0 0.0; -0.44646358 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.003904909 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.042519305 0.0 -0.1963489 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.14311323 0.3187877 0.0 0.0 0.0 0.0; 0.41528898 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.21016508 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.016793787 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.05509817 -0.078497976 0.0 0.0; 0.0 0.0 -0.09824118 0.0 0.0 0.26693752 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.31671545 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.27539808 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.30214667 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.06453327 0.0 0.0 -0.0983223 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.25435638 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.014092606 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.04437834 0.0; 0.0 0.0 -0.58533704 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 -0.08677439 0.4420581 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 -0.037024565 0.042097628 0.0 -0.16222069 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 -0.12982188 0.0 -0.0 0.0 -0.0 0.0 0.0 0.26281044 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.397473 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.10430622 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.26625246 0.21474372 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; -0.15596166 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 -0.0 0.0 0.0 0.0 -0.017639026 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.10125846 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.3379613 0.0 -0.11225547 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0692341 0.0; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.057863813 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.13913456 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.20769718 0.16152158 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.3368593 0.0 0.0 0.0 -0.044632792 0.0 0.0 0.0; -0.32430565 0.0 0.0 0.0 0.001297202 0.0 0.0 0.0 -0.4732126 0.0 0.0; 0.0 0.0 0.0 0.0 -0.3623501 0.0 0.0 -0.1937836 0.053913638 0.0 0.0; 0.19969466 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.4139523 -0.2938606 0.0 0.051682234 0.0 0.0; 0.2903308 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.3315463 0.0 0.0 0.0 0.0 0.0 0.0 -0.25699347 0.0 0.0 0.0; 0.0 0.0 0.27608067 0.0 0.0 -0.2309003 0.0 0.0 0.13415271 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 -0.17517582 0.0 0.0 0.3553512 0.0 0.0 0.0; 0.0 0.0 0.0 0.10580832 0.0 0.0 0.0 0.0 -0.05379232 0.0 0.0; -0.09086647 0.0 -0.07244812 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.2873743 0.0 0.15609993 0.0 0.2812739 0.0 0.0; 0.0 -0.36385432 0.0 0.0 0.0 0.0 0.08682984 0.0 0.0 0.0 0.0; 0.0 0.0 0.20737626 0.0 0.0 0.2806918 0.0 0.0 0.0 0.26197144 0.0; 0.0 -0.119822666 0.0 0.0 0.33828378 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.40880874 0.0 0.0 0.0 -0.19014236 0.22482547 0.0 0.0; -0.0 0.0 -0.015352026 0.0 0.0 0.0 0.0 -0.3825375 0.0 -0.24024552 0.0; 0.0 0.0 0.0 0.0 0.0 0.23300645 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.26382095 0.15234843 0.0 -0.040536396 0.0 -0.11792458 0.0 0.0; 0.0 0.0 0.0 0.38329083 -0.19899073 0.0 0.0 0.095868975 0.16883878 0.0 0.0; 0.39116988 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 -0.21683964 0.0 0.0; -0.33336556 0.0 0.0 0.0 0.0 0.074039996 0.0 0.14601395 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.14257759 0.0 -0.2998564 0.0 0.0; -0.27442315 0.0 -0.11910628 0.0 0.0 -0.010309041 0.0 0.0 0.0 0.0 0.0; -0.11446944 0.0 0.0 -0.12092786 0.20131299 0.0 0.008784488 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.071092606 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.17315939 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 -0.4264732 0.0 0.0 0.0 0.0 0.25475416 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.18984802 0.0 0.0 0.19665737 0.0 -0.13553634 0.3883056 0.0 -0.19951366 0.0 0.0; 0.0 0.0 0.0 -0.2412219 0.23238456 0.0 0.0 0.0 0.42672545 0.0 0.0; 0.0 0.01802656 0.0 0.0 0.0 0.0 0.0 0.24312669 0.0 0.0 0.0; 0.17555425 0.0 -0.006809972 0.0 0.0 -0.07912454 0.0 0.11314732 -0.014095258 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.2342341 0.0 0.30495012 -0.051230863 0.0 0.0 0.12325435 0.16660726 0.0 0.0; 0.0 -0.0022900365 0.16084848 0.0 0.045063693 0.0 -0.13010763 0.0 0.08766489 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.09035304 0.0 0.0 0.0 0.0 0.0 -0.0702894 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.197426 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.031572428 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.11915171 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.2273439 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.21878059 0.0 -0.11130371 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.12165506 0.0 -0.1324719 0.0 0.0 -0.13849677 0.32800412 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.031590726 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.22531855 0.0 0.23983122 0.0 -0.308955 -0.16197915 0.0 0.27081695 0.0 0.0; 0.0 0.0 0.0 -0.30842757 0.088780746 0.0 0.0 -0.009594947 -0.10746981 0.0 0.0; 0.0 -0.056893468 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.29597896 0.0 0.0 0.0 0.0 0.15722835 -0.70405275 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.33726093 0.0 0.1899477 -0.21648611 0.0 0.10338904 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.383646 0.0 0.11186661 0.0 -0.22286521 0.09634888 0.0 0.26747194 0.0 0.0; 0.0 -0.27240717 -0.062025093 0.0 0.0 0.22881925 0.0 -0.028777644 0.060871154 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.13672854 0.0 -0.057787992 0.0 0.0 -0.06987709 0.0 0.26367244 -0.0035662167 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.4324851 0.0 0.19178973 0.0 -0.07548267 0.0 0.17893562 -0.34021413 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.13173363 0.0 0.0 0.0; 0.0 -0.036055602 0.0 0.42830065 0.0 -0.02889482 0.0 0.0 -0.5262276 0.0 0.0; 0.0 0.0 0.40902084 0.0 0.0 -0.0034009367 0.0 -0.114008516 -0.091660656 0.0 0.0; 0.0 0.21908711 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.042856917 0.1113394 0.0 0.0 0.34458968 0.0 0.3195927 0.24627988 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.1896294 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.23741592 0.0 0.0 0.0 -0.048356585 -0.21143842 0.0 0.0 0.0559325 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.38803768 0.0 0.06849829 0.0 0.2734257 0.1311718 0.0 -0.019447006 0.0 0.0; 0.0 -0.18349588 0.0 0.0 0.0 0.37207812 0.0 0.0 -0.080006674 0.0 0.0; 0.0 0.0 0.021640062 0.0 0.0 0.0 0.0 0.65585214 0.0 0.0 0.0; 0.0 -0.22395952 0.05813359 0.0 0.0 0.1412327 -0.17553683 0.0 0.25300908 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.54415447 0.0945877 0.0 0.0 -0.18263727 0.10842237 0.0 0.0; 0.0 0.0 -0.06137091 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.24258241 0.0 0.0 0.0 0.0 0.06187088 -0.013203166 0.0 0.17992166 0.0 0.0; 0.21844092 0.0 0.18574876 0.0 0.0 -0.3009768 -0.09294749 0.0 -0.0928661 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.10646074 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.49450338 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.07894419 0.0 0.0 0.0 0.0 0.0 0.0 -0.7012967 -0.14108041 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.04426661 0.18566284 0.0 -0.108360626 0.0 -0.11705065 0.0 0.0; 0.0 0.0 -0.2792822 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.024284989 0.0 0.0 0.0 0.0 -0.13913451 0.0 0.0 0.0 0.0; 0.37697542 0.0 -0.0 0.0 0.0 0.0 0.18072829 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.15492432 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0 0.0 0.0056114644 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.59804916 0.0 0.0 0.0 0.0 0.0 0.0 -0.44341326 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.026499532 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.3294852 0.0 0.0 0.0 0.07057588 0.0 0.0 0.009690896 0.0 0.057608515 0.0; 0.0 0.0 0.0 -0.24949864 0.00059766416 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 -0.05795815 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.006420493 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.016656794 0.0 0.0 0.0 0.0 0.15787064 0.1277399 0.0 0.0; -0.40196458 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 -0.0 0.0 0.0 0.33495826 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.21770936 0.0; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.3757719 0.0; 0.0 0.0 0.0 0.00822106 -0.019619085 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.038777545 0.0 0.29012936 0.0 0.0 0.0 0.0 -0.01881937 0.0; 0.0 -0.30668986 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.094889075 0.0 0.330581 0.0 0.0 0.0 0.0; 0.0 -0.4232195 0.77022004 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.020265043 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 -0.16658808 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.21841773 -0.100166805 0.0 0.0 0.0 0.0 0.0 0.0 0.07984503 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.026318744 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.23092955 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 -0.18717209 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0072299466 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.11771202 0.0 0.0 0.0 -0.35886592 0.106673 0.0 0.5372845 0.0 0.0; 0.3504431 0.0 0.0 0.0 0.0 -0.00536881 0.23501764 0.0 -0.3522213 0.0 0.0; 0.0 0.0 0.0 0.12989125 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.23544706 0.0 0.031758662 0.0 0.0 -0.29924214 0.23004171 0.0 0.21717256 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.12153498 -0.14911097 0.0 0.0 -0.24881586 0.0 0.0022207163 -0.10119456 0.0 0.0; -0.3055027 0.0 0.24916473 0.0 0.0 0.0 -0.4525425 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.3007746 0.0 0.0 0.007241726 0.0 0.0; 0.0 0.0 -0.26611286 0.0 -0.054190896 0.0 0.0 -0.16479322 -0.07165286 0.0 0.0; 0.0 -0.5360868 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.061925918 0.0 0.0 0.34082234 -0.0 0.0 0.0 0.0 0.1309067 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 -0.08759943 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.12218726 0.11603296 0.0 0.0 -0.7193687 0.19386385 0.0 0.0; 0.0 0.0 0.0 0.0 0.25947347 0.0 0.0 -0.17067862 0.022539087 0.0 0.0; 0.0 -0.2206673 -0.27512348 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.45123994 0.0 0.0 0.0 0.0 0.0 0.087378375 0.0 -0.23345858 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 -0.13966268 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.38151476 0.0 -0.087113984 -0.24065998 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.26275742 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0];
x = Float32[0.0 0.0 0.0 0.0 0.0 0.0 0.7024839 0.37396136 0.48663297 0.04790583 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.913818 0.83895344 1.0747393 0.9157694 0.531085 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.3945374 0.39367357 0.84128493 1.1871763 0.8957511 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.047016922 0.0136369355 0.0 1.1431164 1.1545087 0.052026723 0.0; 0.0 0.0 0.0 0.38046953 0.4257043 0.0 0.0 0.0 0.0 0.96533424 1.3328046 0.04783185 0.0; 0.0 0.0 0.14981532 1.0927224 0.84904623 0.0 0.0 0.0 0.0 1.1674591 1.2666296 0.0 0.0; 0.0 0.0 0.3808476 1.2386377 0.020946616 0.0 0.0 0.0 0.9870167 1.2325084 0.9641549 0.0 0.0; 0.0 0.0 0.88752586 1.255485 0.0 0.0 0.0 0.602117 1.0592078 1.0751748 0.1989526 0.0 0.0; 0.0 0.0 0.76142454 1.3131837 0.37676418 0.15688813 0.21778855 0.94394255 1.2304264 0.9247888 0.0 0.0 0.0; 0.0 0.0 0.15566564 1.1177158 0.96515095 0.85092264 1.0393242 0.9983102 0.74637 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.84622204 0.9451543 0.8906877 0.9758018 0.6879205 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.1503766 0.09604955 0.18730752 0.077307634 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.049386133 0.14898486 0.1325645 0.12316144 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.06316831 0.0359192 0.2073627 0.15487562 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.013903506 0.0 0.0 0.003869682 0.0; 0.0 0.0 0.0 0.0 0.004031089 0.0 0.0 0.0 0.0 0.0 0.04378194 0.009075656 0.0; 0.0 0.038595956 0.0 0.0 0.110722594 0.0 0.0 0.0 0.0 0.0 0.06453198 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.10736046 0.0 0.0; 0.0 0.050960157 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.09612073 0.0107642 0.0 0.0; 0.0 0.16719529 0.0 0.0 0.0 0.0 0.0 0.0 0.045036905 0.12725307 0.0 0.0 0.0; 0.0 0.018823832 0.16039167 0.048514005 0.07420638 0.13816336 0.11856674 0.06499156 0.0790078 0.0 0.0 0.0 0.0; 0.0 0.0 0.20763923 0.106698684 0.22683121 0.120726734 0.1471924 0.05142896 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.116967246 0.46290752 0.15020035 0.39127 0.0063039246 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.01791958 0.07413311 0.17716037 0.2704789 0.4321688 0.4913907 0.07331839 0.0 0.0; 0.0 0.0 0.0 0.0 0.045363553 0.2188691 0.0 0.0 0.0 0.1163124 0.33178633 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.058023147 0.0046733236 0.15924607 0.21055736 0.0 0.32617283 0.010279149 0.0; 0.0 0.0 0.0 0.34182325 0.10577434 0.0 0.0 0.0 0.09940054 0.0550953 0.20891331 0.013612287 0.0; 0.0 0.0 0.38669896 0.21971223 0.16684943 0.0 0.0 0.0 0.39431053 0.062015083 0.11676588 0.005273397 0.0; 0.0 0.15736851 0.012419108 0.11520533 0.012902575 0.0 0.0 0.010272895 0.3088212 0.028943723 0.08461766 0.0 0.0; 0.0 0.0 0.3173272 0.18454988 0.0 0.0 0.0462998 0.52241296 0.07184617 0.0 0.04507792 0.0 0.0; 0.0 0.23527798 0.0 0.4575668 0.13415365 0.058798186 0.48510784 0.04028081 0.028693035 0.10774823 0.0035445134 0.0 0.0; 0.0 0.19170018 0.19495001 0.20696597 0.35712272 0.35763517 0.35003406 0.31231338 0.0 0.052184854 0.0 0.0 0.0; 0.0 0.018351164 0.26245955 0.0 0.0 0.20979413 0.0 0.018182758 0.027539521 0.0 0.0 0.0 0.0; 0.0 0.0 0.18466091 0.021602748 0.101905406 0.0074311616 0.031045182 0.016934896 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.223055 0.610981 0.334049 0.43395233 0.025447773 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.13779442 0.67403555 0.6174175 0.861319 0.7278087 0.27222174 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.66112447 0.39379346 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.31624028 0.018378947 0.0; 0.0 0.0 0.0 0.4165449 0.26005396 0.0 0.0 0.0 0.0 0.0 0.20816734 0.0 0.0; 0.0 0.0 0.2008331 0.7837622 0.40429997 0.0 0.0 0.0 0.0 0.013388442 0.0 0.0 0.0; 0.0 0.0 0.13339566 0.170597 0.0 0.0 0.0 0.0 0.251767 0.0 0.0 0.0 0.0; 0.0 0.0 0.095559485 0.26846334 0.0 0.0 0.053688545 0.27430645 0.031841625 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.2002767 0.28529933 0.09147996 0.19015886 0.27745283 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.29728252 0.7750853 0.58178884 0.32125348 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.16988173 0.0 0.012201735 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.046528116 0.31773975 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.09965427 0.13912198 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.018719614 0.0 0.17287341 0.0 0.0 0.0 0.0; 0.0 0.0 0.13201 0.06661357 0.0 0.0 0.0 0.0 0.0 0.0 0.06324428 0.008649944 0.0; 0.0 0.14296551 0.24774426 0.0 0.0 0.0 0.0 0.09005454 0.17042655 0.0 0.25013503 0.026010608 0.0; 0.0 0.20947358 0.036613178 0.11192453 0.455807 0.0 0.0 0.31407076 0.0 0.0 0.15560241 0.0 0.0; 0.0 0.26078764 0.0 0.124337785 0.0 0.044364017 0.27219743 0.11371686 0.0 0.31386328 0.3772886 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.20470855 0.11657967 0.0 0.0 0.40587434 0.040198673 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.42264548 0.43975377 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.15251829 0.46569955 0.28979808 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.080085404 0.24054296 0.42170867 0.4067143 0.19206034 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.093299285 0.31834987 0.20411357 0.23230363 0.02423195 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.025553307 1.1602402 1.7025049 1.4447716 1.3899046 0.5747102 0.26791364 0.0 0.0; 0.0 0.0 0.0 0.0 0.10024783 1.1410034 2.1171725 2.2083209 2.0551038 2.207937 0.95654917 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.12822396 0.41262966 0.8423072 1.8809403 2.1475792 1.5391741 0.025594128 0.0; 0.0 0.0 0.07250008 0.09229124 0.19609931 0.0 0.0 0.0 0.6263544 2.338801 2.0681045 0.055806275 0.0; 0.0 0.06782978 0.2945273 1.4902318 0.69475114 0.0 0.0 0.04936165 0.64583534 2.429976 1.9605769 0.036394138 0.0; 0.0 0.15742321 1.6663145 2.147428 0.73921597 0.0 0.0 0.022755941 1.968206 2.3461158 1.420795 0.0 0.0; 0.0 0.33443734 1.9896288 1.699648 0.0 0.02436478 0.06670875 1.1097108 2.3570192 2.3770976 0.80154437 0.0 0.0; 0.0 0.27953306 2.0475855 1.939088 0.19201088 0.07474211 1.0221859 2.1813498 2.3856142 1.5711644 0.04068174 0.0 0.0; 0.0 0.37895662 1.7485461 1.9356099 1.1353045 0.65454715 2.1478224 2.4389076 2.2181778 0.49035755 0.0 0.0 0.0; 0.0 0.038260218 1.289968 2.3249414 2.3662524 2.260509 2.29736 1.9944583 0.29999456 0.0 0.0 0.0 0.0; 0.0 0.0 0.6129594 1.7184463 1.6117395 1.7071656 1.0371267 0.1943683 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.6117875 0.7307108 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.6229149 1.1163086 0.7355085 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.5280442 1.1335357 1.014561 0.00076533045 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.1792929 1.1653715 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.6604 1.2822243 0.22348286 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.98053044 1.3009474 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.2867887 1.0242783 0.39522204 0.75657046 0.5374824 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.2748239 0.48383036 0.90361154 1.0315095 1.0822442 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.9406018 1.0839255 0.98701483 1.1871325 1.1367222 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.90984726 1.0608437 1.0538249 1.1890815 0.19989704 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.93181914 0.6047635 0.68532133 0.38129148 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.22661282 0.13195685 0.03621698 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.07828648 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.08398236 0.059924934 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.097370595 0.123735316 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.08692577 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.04475017 0.016867382 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.06878056 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.04275753 0.0 0.0 0.0 0.0 0.09898806 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.027091008 0.0 0.0 0.0 0.027629757 0.06490143 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.05720674 0.17401066 0.13496688 0.09579416 0.08636578 0.03109658 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.22371845 0.0003374215 0.21203603 0.036388617 0.0029900558 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.4886824 0.3548851 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.027076768 0.37401706 0.062228527 0.061115548 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.30126974 0.09059537 0.0019092759 0.0069781383 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.2353742 0.12783714 0.040253434 0.052549772 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.31625384 0.066860855 0.043798696 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.12580821 0.0013172522 0.12938529 0.008219929 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.20292553 0.0 0.067412384 0.26105148 0.41224334 0.19649868 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.22818528 0.0 0.45159137 0.09472399 0.08667154 0.39980012 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.1490166 0.0 0.27603456 0.22691335 0.041878782 0.11961979 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.16147424 0.0 0.0 0.26785013 0.018251356 0.046065852 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.18101929 0.0 0.076725446 0.0 0.013882942 0.0102397045 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.19943045 0.0 0.13874826 0.007959642 0.000984587 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.018811898 0.6840951 0.5645306 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.36750543 0.59464324 0.3316575 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.19592309 0.2729594 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.044809375 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.09253567 0.08716695 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.05124336 0.0 0.5082075 0.68803746 0.36278087 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.12052618 0.32622203 0.47299734 0.70625955 0.65710205 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.10360208 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.02954723 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0010820492 0.20502472 0.075271025 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.26481128 0.21164484 0.0 0.32736236 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.25243598 0.27771035 0.0 0.45074815 0.31458268 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.2345332 0.0 0.38216883 0.42136198 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.26604587 0.14984496 0.029130142 0.36017704 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.21574616 0.0 0.19796462 0.053794935 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.110232085 0.0 0.31280544 0.0890845 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.13454832 0.0 0.4115778 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.3013561 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2033575 0.35738197 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.03782718 0.0 0.0 0.46798766 0.5190735 0.1161295 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.5001028 0.07382436 0.28941846 0.011166299 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.00059426297 0.1121927 0.21727917 0.26960295 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.055266455 0.9800046 2.0564952 1.0984424 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.043975458 1.2001834 2.4565797 2.3080342 0.45320588 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.38960013 2.32301 2.467291 0.4784588 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.043975458 1.4687343 2.2195964 0.91899866 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.29485953 2.117754 2.028374 0.06619084 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.34055644 2.0918 1.5327034 0.23345916 0.29724312 0.22367544 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.78804266 1.887177 0.97028404 2.0918763 2.0765097 1.269057 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.9611712 2.1992297 2.4530199 2.3705344 2.369037 1.7792461 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.6671213 2.3467343 2.4720352 2.0855837 2.2112226 0.8011193 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.53999573 2.217857 2.306669 2.3036492 1.479879 0.11752501 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.64559823 1.0840514 0.69714725 0.49816042 0.011300483 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0];
w = Float32[0.2960441 0.111418106 0.060419407; 0.17770372 0.14079462 0.15887481; 0.18907367 -0.16234794 0.17205341;;; -0.13349609 0.059268158 0.14664102; 0.23368506 0.16117053 0.20016909; -0.0008412939 -0.019029753 0.023617527;;; -0.13384163 -0.14346288 -0.1393783; -0.3743223 0.37621766 -0.31452104; -0.18737818 0.0050915703 0.40118456;;; 0.33077124 -0.11322178 -0.16582306; 0.08322287 0.15553156 -0.12731376; 0.07154577 0.59240407 -0.16393593;;; 0.0961527 -0.11572408 -0.27439263; -0.2603522 0.2475533 0.1852849; 0.039479923 -0.076529376 -0.008782608;;; 0.15987548 0.14487784 -0.20540781; -0.090020254 0.07815908 -0.11656536; 0.24148916 0.123776756 -0.013628215;;;; -0.18501134 -0.5018476 0.1364615; -0.033195734 -0.059265275 -0.12340808; 0.03279507 0.030364413 -0.062295433;;; 0.10242772 0.07022454 0.21112; 0.13173018 -0.01931001 0.19177276; 0.14826803 0.27158484 -0.09574231;;; -0.103504315 -0.23975907 0.022229772; 0.29308864 -0.31528187 0.17985724; 0.019600185 -0.11988832 0.13303092;;; -0.1440467 -0.37177914 -0.07335549; -0.07964619 0.18814577 0.30254102; -0.15585509 -0.037972923 0.15434241;;; 0.21757427 0.1493668 -0.43167758; 0.06206128 -0.24268627 0.12340345; 0.10159719 0.27610138 -0.32440266;;; -0.18050247 -0.12446522 -0.15579449; -0.17349248 0.22638118 0.17040627; -0.06882828 -0.2464726 0.48966035;;;; 0.032532074 -0.023223227 0.028135454; 0.10584263 0.37556973 -0.27503803; 0.08007367 -0.014648067 -0.20753586;;; 0.17736532 -0.02143073 0.25086683; -0.09793015 -0.26255456 -0.12905733; -0.03844144 -0.29277217 -0.21530282;;; -0.035122722 -0.31351343 -0.15625529; -0.25264773 0.1820678 0.10031157; -0.27853096 0.2775023 0.42040953;;; -0.15394558 -0.019259363 0.27723357; 0.14209418 -0.09762835 -0.017030459; -0.1748959 -0.1854027 0.20193024;;; 0.053943474 0.0019853031 -0.30946; 0.02569863 -0.30160666 -0.21075335; 0.12672381 0.16118068 -0.27732012;;; 0.14221564 -0.35081193 -0.03159928; 0.066303305 -0.07110759 -0.16112953; 0.18001214 -0.16219966 -0.010493229;;;; 0.06374236 -0.11735779 0.17702304; -0.29021615 -0.050297696 0.22225729; -0.35975787 -0.027783519 0.08350159;;; -0.121532515 -0.1437047 -0.25527862; 0.12492706 0.093595676 0.0877293; -0.2781678 -0.05474469 -0.05044482;;; -0.089153305 0.15517852 -0.31376806; 0.104027346 0.023436652 0.19947311; 0.010470727 0.061271884 -0.49059668;;; -0.20311777 -0.17262228 -0.09114029; -0.2760243 -0.25582096 0.15906595; -0.066743 -0.46341977 -0.016560892;;; -0.060133208 -0.23535502 -0.024656134; 0.09745767 -0.16524819 0.3225349; -0.43826315 0.02562342 -0.10293539;;; 0.00787415 -0.012549186 -0.15232417; -0.3190784 -0.08864445 0.07531635; 0.057686575 0.2804737 0.22664623;;;; -0.07713373 0.07829692 0.029764323; 0.20172861 0.24266954 0.26024637; 0.08879657 0.09821315 0.061714884;;; -0.057199515 -0.06155156 0.4218559; 0.07331105 -0.026877264 -0.4089807; -0.074377276 0.012273083 0.13384485;;; -0.38058874 -0.19863698 -0.0035578944; 0.09134012 -0.013197209 0.066105805; 0.21941046 -0.038204145 -0.2252429;;; 0.31430575 -0.08551885 0.07776549; -0.4893492 -0.057713516 0.46964762; -0.2937467 0.32734746 -0.17586322;;; 0.22087777 -0.14655845 0.0961565; -0.13657235 0.35338503 -0.1513152; -0.08373559 0.025258316 -0.037407767;;; 0.14634459 0.10477101 0.052650984; 0.08023411 -0.20191658 0.15784113; 0.50030804 0.18490331 0.026398752;;;; 0.12502794 0.24942584 -0.059281297; 0.20331834 0.43808994 0.044574223; -0.11406282 0.25983557 0.02928696;;; -0.07471862 -0.10680411 0.4733718; 0.07784771 0.22030471 0.19235015; -0.097382426 -0.21797614 -0.27371573;;; 0.19578451 -0.16234678 -0.04979607; 0.15080474 0.32766116 0.3350032; 0.29100034 -0.18732655 -0.16610901;;; -0.1735875 -0.09669232 0.036246907; 0.16105351 0.01524193 0.13218386; -0.0070821703 -0.36703375 0.07152267;;; -0.037466418 0.12137482 0.24356191; -0.14597577 -0.17833084 -0.2849881; 0.22209632 -0.017434767 0.20813318;;; 0.066320755 0.14026296 0.1755846; 0.34621808 -0.09216782 0.1618269; -0.17755938 -0.1261339 0.3179875;;;; -0.08989663 0.33881766 -0.16007112; -0.22741039 -0.019246599 0.15831085; -0.2575929 0.0071016536 0.05472914;;; -0.12973835 -0.05470754 0.085103594; 0.04473279 -0.18376277 -0.17163287; -0.002223049 -0.051479645 0.0012604808;;; -0.3450564 -0.03766715 -0.33915076; 0.13909505 0.054624945 0.08095364; -0.005161413 0.16282125 0.14670944;;; 0.12544005 0.01231547 0.15847863; 0.2635919 -0.13353993 0.36272755; 0.087062076 0.21166635 -0.061734356;;; -0.057979632 0.12484703 0.011279334; 0.3187766 0.075600296 0.14169012; -0.21386455 -0.08302145 -0.050783318;;; 0.3252191 -0.038426936 0.19923727; 0.11715561 -0.29249743 -0.10526862; 0.20977305 0.49560124 -0.11074418;;;; -0.070868246 0.2989184 -0.021860158; 0.14750893 0.022390867 -0.1673016; 0.23230949 0.12457042 0.27895197;;; 0.26734343 -0.20439851 -0.29521325; 0.15159248 -0.10931299 -0.06471817; 0.22746634 -0.2576135 0.5312447;;; 0.34235042 -0.1419672 -0.14499076; 0.06054748 0.013098666 0.46927348; 0.027008578 -0.24567385 0.17514727;;; -0.016410237 0.13800983 0.311671; 0.28729555 -0.27811 0.09542504; -0.108946696 -0.13398571 0.06440614;;; -0.17438737 0.24925423 -0.06299011; 0.08006524 -0.102395795 0.11736292; -0.061629456 -0.120828025 -0.21055403;;; 0.1710203 -0.31161055 -0.18735167; -0.09807411 0.4459311 -0.22391868; 0.07561545 0.16240786 -0.2390652;;;; -0.0861018 -0.61804795 0.26165327; -0.104731 -0.16223292 0.23340371; -0.33034143 0.16457711 -0.029518232;;; -0.011202748 -0.10577551 -0.2504469; -0.07724738 -0.12746224 0.26280668; 0.0043004514 0.077701464 -0.2021106;;; 0.1249029 0.22228606 -0.03274191; -0.20470499 0.007915322 0.21182826; -0.20129903 -0.07883954 0.1047755;;; -0.034731265 -0.06827126 -0.24449372; -0.25888062 0.15017 -0.09789805; 0.09881266 0.01744669 -0.11119189;;; -0.040175155 -0.3784317 0.10595681; 0.08331488 -0.3688626 -0.53431344; 0.27519742 -0.08197422 0.1871619;;; -0.08393871 -0.177044 -0.20151941; 0.5216566 0.03994574 -0.24035357; -0.20596762 -0.101699926 -0.028122244;;;; -0.08408671 -0.4309965 -0.16341875; -0.25342238 0.062758416 -0.026333679; -0.19613807 0.111440375 0.23561847;;; -0.31230855 -0.42451176 0.12752436; -0.071124606 -0.25800827 0.100179575; 0.2005591 -0.29734227 0.34354362;;; -0.18008648 -0.054225978 0.38231227; 0.22249281 0.34637973 -0.10459437; -0.30952713 0.0943418 -0.24401377;;; -0.2094655 0.04347092 -0.16068658; 0.21253741 0.18162587 0.111536495; 0.018626627 -0.27955675 0.2376918;;; 0.1173759 0.03448805 -0.10867351; -0.29966062 0.23842323 -0.041477848; 0.16589904 -0.1377726 -0.26018885;;; 0.08899052 -0.19468556 -0.14165412; -0.067954056 -0.1093858 0.17397392; 0.01975474 0.16994041 0.07281881;;;; -0.19929275 -0.030734325 0.15269162; -0.2237056 0.07531421 -0.20523536; -0.17838272 0.0955053 0.08355754;;; 0.19949287 -0.3143494 -0.048280254; 0.03563079 0.23299764 0.006331266; -0.28459817 -0.15420969 -0.06389524;;; -0.3258877 0.27254832 0.18485771; 0.14840262 0.085267596 -0.1274465; -0.19189955 0.03848765 0.45244023;;; -0.14617044 -0.32841134 -0.11458603; 0.014426745 -0.18841083 0.38842657; -0.21451603 -0.023593398 -0.1367498;;; -0.061822243 0.46621808 -0.13356048; -0.05291281 -0.03132733 -0.32685736; 0.030487243 -0.19138455 0.07941134;;; -0.3217983 -0.14114253 -0.009285473; 0.30364543 0.011280674 0.20578769; -0.053509828 -0.045276333 -0.10787562;;;; 0.17140536 0.10998141 -0.27693063; 0.04159409 0.09552857 -0.3083664; 0.2509009 -0.16674422 0.2715514;;; -0.25238332 0.03144278 -0.06793796; 0.24989843 -0.010943731 -0.22215822; 0.052133083 -0.514625 0.050451525;;; -0.19814375 -0.46190342 0.26085097; -0.1353483 0.12587525 0.19985119; 0.35497203 -0.18794687 0.25051442;;; -0.039527606 0.10292178 0.061134405; -0.1718076 0.16613112 -0.1768595; 0.24302395 0.41874632 0.2893839;;; 0.20302151 0.13232446 -0.1027738; -0.10516542 -0.03918797 0.10122553; 0.33903834 -0.106636204 0.18255593;;; 0.04527327 0.07685093 0.15391664; -0.19081439 0.26250747 0.19692656; -0.5239402 -0.16781174 -0.4941832;;;; -0.23401299 0.15363865 0.1351295; -0.04321471 0.24311677 0.055175733; -0.079230934 -0.15453355 -0.16824965;;; 0.44954053 -0.15271842 0.010085229; -0.021147853 -0.030766169 0.040853236; 0.1229757 -0.15910581 -0.059540976;;; 0.40011463 -0.51326406 -0.0128572825; -0.17922334 0.02396191 -0.17640088; -0.07196925 0.08384376 -0.013414534;;; 0.108110525 -0.20552197 -0.0799298; -0.07223762 0.1034506 0.023793427; -0.22661312 -0.028116897 -0.027126579;;; -0.42725116 0.24330647 0.21449482; 0.05262759 -0.0784973 0.24016702; 0.15515503 0.0016307709 0.27052978;;; 0.28625467 -0.12179351 -0.2442141; -0.054535348 -0.15453765 0.059011348; -0.15124421 -0.30620453 0.19068305;;;; 0.033986215 -0.014089408 -0.23575716; 0.076355256 0.40448546 -0.028241752; -0.25016463 -0.05257351 0.021255424;;; 0.32597533 0.031241143 0.036040824; -0.05040468 -0.116196 0.00019899952; -0.17527637 -0.049230017 0.12940045;;; 0.21847914 -0.21157086 -0.3799399; -0.19211052 0.10920056 -0.037067708; 0.30646247 -0.2485291 -0.02367381;;; -0.042462695 -0.2903074 0.21462563; -0.19774127 0.2484361 -0.1445145; 0.04823768 0.018797502 -0.023630751;;; -0.24138747 0.0060263434 -0.13349238; 0.09480944 -0.22589114 -0.10299342; -0.087907754 0.409123 -0.25563315;;; 0.041950163 -0.15631694 -0.39972576; 0.15582007 -0.21364895 0.21985257; 0.27561778 0.34985474 0.14027534;;;; 0.108420126 -0.16199157 0.10476513; -0.0897881 -0.03807054 -0.22312157; 0.14197476 -0.21897955 0.34654388;;; -0.085977666 -0.27313077 -0.06267007; 0.051328834 0.24373488 -0.16352099; 0.0858128 0.14201657 -0.13043292;;; 0.26280925 -0.18252693 0.02737608; -0.018303527 -0.041822698 0.16170755; -0.021939484 0.066071905 -0.04148299;;; 0.2624021 0.08232013 0.046253946; -0.23195294 -0.17704348 -0.20016094; -0.4979984 0.0030149352 -0.24303107;;; 0.19410527 0.24212852 -0.29089448; -0.5473085 -0.50948936 0.267508; 0.03466059 0.24743183 -0.03744423;;; -0.048880566 0.1887267 0.20196238; 0.07319751 -0.18055718 -0.28769; 0.093184814 -0.10162357 0.29118487;;;; -0.039678853 0.05738231 -0.25534865; -0.43570837 0.068097435 0.017311903; -0.05554346 0.4040946 0.05375871;;; -0.06747358 0.14294383 0.1209641; -0.027304268 -0.22928528 0.03553192; 0.027479146 -0.19665293 -0.10321096;;; 0.25762305 -0.39518824 0.19714986; 0.04074652 0.19865908 -0.3229461; 0.23679538 0.11057849 -0.5662169;;; -0.22486027 -0.26239726 0.16333725; -0.04857991 0.247622 -0.21413146; 0.09376545 -0.09250514 0.013400458;;; 0.047771722 -0.1323898 0.15110826; 0.2459936 -0.17620137 0.07654275; -0.09651109 -0.0453217 -0.17001072;;; -0.0399055 -0.072917394 -0.06997162; 0.09203763 0.10449474 0.013086869; 0.16443078 0.06832409 0.26274332];
b = Float32[-0.05037788, 0.002298597, -0.011481969, 0.0071880803, -0.07189355, -0.013801541, -0.052262988, 0.0014741025, -0.0065286574, -0.012910124, -0.03790738, -0.01459733, 0.0007291272, -0.06687447, -0.03561023, -0.044976227];

In [10]:
size(x)

(13, 13, 6, 2)

In [11]:
function maxPool(x::Array{Float32, 4}, kernel_size::Tuple{Int64, Int64})
    h, w, c, n = size(x)
    kernel_h, kernel_w = kernel_size
    out_h, out_w = div(h, kernel_h), div(w, kernel_w)
    output = Array{Float32}(undef, out_h, out_w, c, n)
    @fastmath @inbounds for m=1:n
        for i = 1:c
            for j = 1:out_h
                j_start = (j - 1) * kernel_h + 1
                j_end = j * kernel_h
                @views for k = 1:out_w
                    k_start = (k - 1) * kernel_w + 1
                    k_end = k * kernel_w
                    @inbounds window = x[j_start:j_end, k_start:k_end, i, m]
                    val, idx_flat = findmax(window)
                    @inbounds output[j, k, i, m] = val
                end
            end
        end
    end
    return output
end

maxPool (generic function with 1 method)

In [12]:
@benchmark maxPool(x, (2,2))

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.060 μs … 121.320 μs  ┊ GC (min … max): 0.00% … 89.91%
 Time  (median):     6.660 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.905 μs ±   1.411 μs  ┊ GC (mean ± σ):  0.16% ±  0.90%

      ▅██▅▂                                                    
  ▁▂▄▆██████▆▅▄▄▄▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  6.06 μs         Histogram: frequency by time        10.8 μs <

 Memory estimate: 1.81 KiB, allocs estimate: 2.

In [13]:
@benchmark maxpool(x, (2,2))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.240 μs … 168.880 μs  ┊ GC (min … max): 0.00% … 97.35%
 Time  (median):     1.350 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.592 μs ±   3.243 μs  ┊ GC (mean ± σ):  4.38% ±  2.15%

  ▁▆█▇▅▃▂     ▁▂▂▂▂▃▃▃▃▂▂▁▁                                   ▁
  █████████▇▆██████████████▇▇▇█▇▆█▇▇▇▆▆▅▅▄▃▄▅▆▅▆▅▆▅▅▅▅▄▄▆▅▆▅▅ █
  1.24 μs      Histogram: log(frequency) by time      3.27 μs <

 Memory estimate: 2.50 KiB, allocs estimate: 13.

In [52]:

# This function computes the gradient of the loss with respect to the inputs, weights, and biases of a convolution layer
function conv_backward(x, w, b, g)
    H, W, C, N = size(x)
    HH, WW, _, F = size(w)  # Correctly extract the number of filters and ignore the redundant channel size
    H_prime = H - HH + 1
    W_prime = W - WW + 1
    m = size(x, 1)
    dx = zeros(Float32, size(x))
    db = reshape(sum(g, dims=(1, 2, 4)), F)
    dw = zeros(Float32, size(w))
    dout_reshaped = reshape(g, size(g, 1) * size(g, 2), size(g, 3), size(g, 4))
    dout_reshaped = permutedims(dout_reshaped, (2, 1, 3))
    X_col = im2col(x, HH, WW, H_prime, W_prime)
    dw_col = dout_reshaped * X_col'
    dw2 = reshape(dw_col, size(w))
    @views for i=1:N
        im = x[:, :, :, i]
        im_col = im2col_back2(im, HH, WW)
        grad_i = g[:, :, :, i]
        println(size(grad_i))
        grad_col = reshape(grad_i, H_prime*W_prime, F)
        println(size(grad_col))
        dbias = transpose(reshape(grad_i, :, F))
        
        #Wagi
        dfilter_col = im_col * grad_col
        dw += reshape(dfilter_col, HH, WW, C, F)

        # Input
        dbias = transpose(reshape(grad_i, :, F))
        dim_col = dbias' * transpose(reshape(w, HH*WW*C, F))
        col2im!(dx[:, :, :, i],dim_col, H_prime, W_prime, HH, WW, C)
    end
    println(dw2 == dw)
    println(dw2)
    println(dw)
    return tuple(dx, dw, db)
end

# Converts a batch of images into column format for easier multiplication in convolution
function im2col_back2(x::AbstractArray, HH::Int64, WW::Int64)
    H, W, C = size(x)
    H_prime = H - HH + 1
    W_prime = W - WW + 1
    col = zeros(Float32, HH*WW*C, H_prime*W_prime)
    for i=1:H_prime
        for j=1:W_prime
            @inbounds patch = @view x[i:i+HH-1, j:j+WW-1, :]
            @inbounds @views col[:, (i-1)*W_prime+j] = patch[:]
        end
    end
    return col
end

# Converts column format back to the original image shape
function col2im!(dx::AbstractArray, col::AbstractArray, H_prime::Int64, W_prime::Int64, HH::Int64, WW::Int64, C::Int64)
    for i=1:H_prime*W_prime-1
        row = @view col[i, :]
        h_index = div(i, W_prime) + 1
        w_index = mod(i, W_prime) + 1
        @inbounds dx[h_index:h_index+HH-1, w_index:w_index+WW-1, :] += reshape(row, HH, WW, C)
    end
    return dx
end

col2im! (generic function with 1 method)

In [53]:
dx, dw, db = conv_backward(x, w, b, g);
dx

(121, 16, 2)


MethodError: MethodError: no method matching *(::Array{Float32, 3}, ::Adjoint{Float32, Matrix{Float32}})

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:587
  *(!Matched::ChainRulesCore.ZeroTangent, ::Any)
   @ ChainRulesCore C:\Users\mbili\.julia\packages\ChainRulesCore\zgT0R\src\tangent_arithmetic.jl:104
  *(::Any, !Matched::ChainRulesCore.NotImplemented)
   @ ChainRulesCore C:\Users\mbili\.julia\packages\ChainRulesCore\zgT0R\src\tangent_arithmetic.jl:38
  ...


In [36]:
@benchmark conv_backward(x, w, b, g)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  194.800 μs …  2.953 ms  ┊ GC (min … max): 0.00% … 90.46%
 Time  (median):     211.500 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   234.800 μs ± 99.559 μs  ┊ GC (mean ± σ):  3.00% ±  6.42%

   ▅▇█▇▅▄▄▃▃▃▃▄▄▃▂▂▂▂▂▂▁▁▁▁▁                                   ▂
  ▇████████████████████████████▇▇▇▇▇▇▇▇▆▇▆▇▇▇▆▆▅▅▆▅▅▆▅▄▆▄▅▄▅▄▄ █
  195 μs        Histogram: log(frequency) by time       426 μs <

 Memory estimate: 312.45 KiB, allocs estimate: 517.

In [47]:
using BenchmarkTools
@benchmark conv_backward(x, w, b, g)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  141.900 μs …  2.419 ms  ┊ GC (min … max): 0.00% … 91.76%
 Time  (median):     151.900 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   162.154 μs ± 67.694 μs  ┊ GC (mean ± σ):  2.78% ±  5.91%

   ▅▇▇██▇▅▅▄▃▂▁▁▃▃▃▃▃▂▁▁▁▁                                     ▂
  ▇███████████████████████████▇▇▇▇▇▇▆▇▆▆▆▆▅▅▄▆▄▅▅▆▆▅▆▅▄▅▂▃▄▃▃▄ █
  142 μs        Histogram: log(frequency) by time       256 μs <

 Memory estimate: 257.88 KiB, allocs estimate: 538.

In [10]:
using BenchmarkTools
@benchmark conv_backward(x, w, b, g)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  147.800 μs …   2.676 ms  ┊ GC (min … max): 0.00% … 90.65%
 Time  (median):     163.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   181.051 μs ± 100.635 μs  ┊ GC (mean ± σ):  3.46% ±  6.08%

   ▁▇█▅▁                                                         
  ▃█████▅▄▃▃▃▅▅▆▅▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  148 μs           Histogram: frequency by time          320 μs <

 Memory estimate: 257.88 KiB, allocs estimate: 538.

In [194]:
conv_backward_og(x, w, b, g)

(Float32[0.0 0.0 … 0.0122897215 0.0; 0.0 0.0 … 0.013684789 0.0; … ; 0.0 -0.014087212 … 0.0012090683 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.003014974 0.0; 0.0 0.0 … 0.009859013 0.0; … ; 0.0 0.006762931 … 0.0015461952 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … -0.011576133 0.0; 0.0 0.0 … -0.008869064 0.0; … ; 0.0 0.049001433 … 0.0017700926 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … -0.0027130435 0.0; 0.0 0.0 … 0.007848731 0.0; … ; 0.0 0.030769112 … 0.010799806 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0045609307 0.0; 0.0 0.0 … -0.004492221 0.0; … ; 0.0 -0.022294285 … -0.018678991 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … -0.006830565 0.0; 0.0 0.0 … -0.008739274 0.0; … ; 0.0 -0.031669203 … 0.022821872 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.026078537 -0.06395636 … -0.09035085 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.00033949566 -0.009854607 … -0.17945124 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.015891388 -0.11509025 … -0.04

In [190]:
using BenchmarkTools
@benchmark conv_backward_og(x, w, b, g)

BenchmarkTools.Trial: 1416 samples with 1 evaluation.
 Range (min … max):  3.236 ms …   5.701 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.445 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.524 ms ± 272.329 μs  ┊ GC (mean ± σ):  1.89% ± 4.82%

     ▅▇█▇▆▂▃▃▂▁▄▁▂                                             
  ▃▄██████████████▆▅▃▄▄▃▂▁▂▂▂▁▁▁▁▁▁▂▂▂▂▃▂▂▂▃▁▂▂▂▂▁▂▂▂▂▂▁▁▁▂▁▁ ▃
  3.24 ms         Histogram: frequency by time        4.45 ms <

 Memory estimate: 4.38 MiB, allocs estimate: 23238.

In [ ]:
xxx

In [15]:
println(size(x))
println(size(w))
println(size(b))
println(size(g))

(13, 13, 6, 2)
(3, 3, 6, 16)
(16,)
(11, 11, 16, 2)


In [188]:
function conv_backward_og(x, w, b, g)
    H_R, W_R, F, N = size(g)
    H, W, C, N = size(x)
    HH, WW, C, F = size(w)
    dx = zeros(Float32, size(x))
    dw = zeros(Float32, size(w))
    db = zeros(Float32, size(b))
    for n=1:N
        for depth=1:F
            for r=1:H_R
                for c=1:W_R
                    gje = g[r, c, depth, n]
                    @views dx[r:r+HH-1, c:c+WW-1, :, n] += w[:, :, :, depth] * gje
                    @views dw[:, :, :, depth] += x[r:r+HH-1, c:c+WW-1, :, n] * gje
                end
            end
        end
    end
    for depth=1:F
        db[depth] = sum(@view g[:, :, depth, :])
    end
    return tuple(dx, dw, db)
end


conv_backward_og (generic function with 1 method)

In [43]:
using BenchmarkTools
@benchmark conv_fast2(test_n, jj)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  45.400 μs …   5.527 ms  ┊ GC (min … max): 0.00% … 97.77%
 Time  (median):     60.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   65.371 μs ± 151.225 μs  ┊ GC (mean ± σ):  7.19% ±  3.09%

  █▂           █▆▃▁▁                                            
  ██▆▇▆▃▂▂▂▁▂▃▇█████▇▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  45.4 μs         Histogram: frequency by time          107 μs <

 Memory estimate: 56.12 KiB, allocs estimate: 9.

In [27]:
sum(conv_noweP(test_n, jj))

178.95865947494167

In [53]:
@benchmark conv(test_n, jj)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   7.533 μs …   6.016 ms  ┊ GC (min … max):  0.00% … 98.21%
 Time  (median):     26.067 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   42.746 μs ± 197.104 μs  ┊ GC (mean ± σ):  23.13% ±  5.17%

  ▂▅▄▃█▆▃▃▂▁▁                                                   
  ████████████▇▇▆▅▅▅▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  7.53 μs         Histogram: frequency by time          146 μs <

 Memory estimate: 43.22 KiB, allocs estimate: 39.

In [32]:
conv_noweP(test_n, jj)

26×26×6×1 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0         0.0         …   0.0        0.0        0.0
 0.0  0.0  0.0  0.0         0.0             0.0        0.0        0.0
 0.0  0.0  0.0  0.0         0.0            -0.150675  -0.0739125  0.0
 0.0  0.0  0.0  0.0         0.0            -0.166691   0.0540174  0.0
 0.0  0.0  0.0  0.0         0.0             0.13342    0.15541    0.0
 0.0  0.0  0.0  0.0         0.0         …   0.249221   0.100895   0.0
 0.0  0.0  0.0  0.0        -0.0118477       0.298495   0.123178   0.0
 0.0  0.0  0.0  0.0        -0.00159272      0.331982   0.107289   0.0
 0.0  0.0  0.0  0.0         0.0578697       0.254556   0.0195904  0.0
 0.0  0.0  0.0  0.0         0.0745503       0.289135   0.114286   0.0
 ⋮                                      ⋱                         ⋮
 0.0  0.0  0.0  0.42533     0.898905        0.0        0.0        0.0
 0.0  0.0  0.0  0.411135    0.600968        0.0        0.0        0.0
 0.0  0.0  0.0  0.0383475   0.356761        0.0 

In [33]:
conv(test_n, jj)

26×26×6×1 Array{Float32, 4}:
[:, :, 1, 1] =
 0.0  0.0  0.0   0.0         0.0         …  0.0       0.0        0.0
 0.0  0.0  0.0   0.0         0.0            0.0       0.0        0.0
 0.0  0.0  0.0   0.0         0.0            0.273018  0.361955   0.0
 0.0  0.0  0.0   0.0         0.0            0.790526  0.730574   0.0
 0.0  0.0  0.0   0.0         0.0            0.904661  0.7261     0.0
 0.0  0.0  0.0   0.0         0.0         …  0.849128  0.679894   0.0
 0.0  0.0  0.0   0.0        -0.0139782      0.800986  0.570775   0.0
 0.0  0.0  0.0   0.0         0.0254724      0.696306  0.348591   0.0
 0.0  0.0  0.0   0.0        -0.00940702     0.688012  0.324396   0.0
 0.0  0.0  0.0   0.0         0.0410519      0.622795  0.0448162  0.0
 ⋮                                       ⋱                       ⋮
 0.0  0.0  0.0   0.16041     0.09789        0.0       0.0        0.0
 0.0  0.0  0.0  -0.13584     0.0153537      0.0       0.0        0.0
 0.0  0.0  0.0   0.100818    0.454757       0.0       0.0    

In [65]:
@benchmark conv_fast2(test_n, jj)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  53.700 μs …   9.857 ms  ┊ GC (min … max): 0.00% … 98.69%
 Time  (median):     69.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   80.306 μs ± 265.364 μs  ┊ GC (mean ± σ):  9.26% ±  2.79%

  ▄▆▇█▆▆▅▅▇▇▆▆▅▄▃▃▂▂▂▂▂▁▁▁   ▁                                 ▂
  ████████████████████████████▇███▇██▇▆▇▇▅▇▆█████▇▇▆▆▆▅▅▅▅▆▂▄▅ █
  53.7 μs       Histogram: log(frequency) by time       165 μs <

 Memory estimate: 57.34 KiB, allocs estimate: 22.

In [32]:
sum(conv_nower(test_n, jj))

378.4857544422375